In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## 데이터 읽어오기

In [2]:
def Read_data(filename):
    df = pd.read_csv("{}.csv".format(filename))
#     df.set_index('Date', inplace=True)
#     df.sort_index(ascending=True, inplace=True)
    df['Name'] = filename # 식별자 추가(나중에 분석 돌리기 위해서 추가함..)
    return df

## 데이터 가공

In [3]:
def Make_Data_Polynomial(filename):
    df = Read_data(filename)
    X = df['Date'].values # 처음부터 마지막 컬럼 직전까지의 데이터 (독립 변수 - 원인)
    y = df['Close'].values # 마지막 컬럼 데이터 (종속 변수 - 결과)
    df = pd.DataFrame(X, columns=['Date'])
    # 'Date' 열의 데이터를 날짜로 변환
    df['Date'] = pd.to_datetime(df['Date'])
    # 'Date' 열의 데이터를 숫자로 변환
    df['Date_numeric'] = pd.factorize(df['Date'])[0]
    X = df['Date'].values 
    X = df['Date_numeric'].values
    X = X.reshape(-1, 1)
    return X, y

## 그래프 그리기

In [4]:
def Polynomial_graph(filename, degree):
    X = Make_Data_Polynomial(filename)[0]
    y = Make_Data_Polynomial(filename)[1]
    poly_reg = PolynomialFeatures(degree=degree)
    X_poly = poly_reg.fit_transform(X)
    lin_reg = LinearRegression()
    lin_reg.fit(X_poly, y)

    x_range = np.linspace(X.min(), X.max(), 100)
    X_range_poly = poly_reg.transform(x_range.reshape(-1, 1))
    y_range = lin_reg.predict(X_range_poly)

    fig = px.scatter(x=X.flatten(), y=y, opacity=0.65)
    fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Polynomial Fit'))
    fig.update_layout(
        title='Polynomial Model',
        xaxis_title='date',
        yaxis_title='value',
        autosize=True # 추가한 부분
    )
    return fig.show()

In [15]:
Polynomial_graph("UK",5)

## 예측

In [6]:
def Polynomial_Predict_days(filename, degree, end):# degree 는 숫자 , 5일 후만 예측
    X = Make_Data_Polynomial(filename)[0]
    y = Make_Data_Polynomial(filename)[1]
    result = []
    poly_reg = PolynomialFeatures(degree=degree) # 그래프의 X 계수를 결정
    X_poly = poly_reg.fit_transform(X)
    lin_reg = LinearRegression()
    lin_reg.fit(X_poly, y) # 변환된 X 와 y 를 가지고 모델 생성 (학습)
    for i in range(int(Make_Data_Polynomial(filename)[0][-1]) + 1, int(Make_Data_Polynomial(filename)[0][-1]) + 1 + end):
        predict_value = lin_reg.predict(poly_reg.fit_transform([[i]]))
        result.append(predict_value)
    return result       

In [17]:
Polynomial_Predict_days("UK", 5, 5)

[array([1330.20166518]),
 array([1330.84785341]),
 array([1331.8331873]),
 array([1333.21628259]),
 array([1335.05969458])]